# Model Initiation

In [1]:
#Libary
import pandas as pd

In [2]:
import ixmp

mp = ixmp.Platform()

2025-02-17 14:11:27,166  INFO at.ac.iiasa.ixmp.Platform:165 - Welcome to the IX modeling platform!
2025-02-17 14:11:27,167  INFO at.ac.iiasa.ixmp.Platform:166 -  connected to database 'jdbc:hsqldb:file:/Users/ilhamrizaldi/.local/share/ixmp/localdb/default' (user: ixmp)...


In [3]:
import message_ix

scenario = message_ix.Scenario(
    mp, model='Indonesia Prototype', scenario='baseline', version='new'
)

# Model Structure

<img src='Coal Messageix Structure.drawio.png' width='1000'>

In [4]:
horizon = list(range(2005,2051,1))
scenario.add_horizon(year=horizon, firstmodelyear=2023)


In [5]:
country='Indonesia'
scenario.add_spatial_sets({'country':country})

In [6]:
scenario.add_set('commodity',['coal','electricity'])
scenario.add_set('level',['primary','secondary','final'])
scenario.add_set('technology',['coal_bal','coal_ppl','coal_t_d','electricity_grid'])
scenario.add_set('mode','standard')

## Demand Parameter

In [7]:
coal_d = [
    57.18083, 60.03523, 63.04894, 66.23524, 69.58086, 73.11234, 76.81642, 79.94961, 
    83.16247, 86.46825, 89.86697, 93.34535, 96.92994, 100.60747, 104.37792, 108.22804, 
    112.18436, 116.23362, 120.36253, 124.58438, 128.88589, 133.26705, 137.72788, 142.2418, 
    146.80884, 151.4157, 156.08894, 160.80201
]#converted to GWa

electricity_d = [
    21.00799087, 21.65068493, 22.32191781, 23.03310502, 23.10844749, 23.16552511, 
    23.34589041, 23.46232877, 23.7956621, 24.1826484, 24.54680365, 24.89155251, 
    25.28652968, 25.64611872, 26.15068493, 26.65410959, 27.21461187, 27.71803653, 
    28.32762557, 28.94406393, 29.51712329, 30.16324201, 30.80022831, 31.4303653, 
    32.12442922, 32.77739726, 33.42694064, 34.06849315
]#converted to GWa

model_horizon = list(range(2023,2051,1))

coal_demand = pd.DataFrame(
    {
        'node':country,
        'commodity':'coal',
        'level':'final',
        'year': model_horizon,
        'time':'year',
        'value':coal_d,
        'unit':'GWa'
    }
)
coal_demand['value'] = coal_demand['value'].round(decimals=2)

electricity_demand = pd.DataFrame(
    {
        'node':country,
        'commodity':'electricity',
        'level':'final',
        'year': model_horizon,
        'time':'year',
        'value':electricity_d,
        'unit':'GWa'
    }
)

electricity_demand['value'] = electricity_demand['value'].round(decimals=2)



In [8]:
scenario.add_par('demand',coal_demand)
scenario.add_par('demand',electricity_demand)

## Engineering Parameters

In [9]:
year_df = scenario.vintage_and_active_years()
vintage_years,act_years = year_df["year_vtg"],year_df["year_act"]

In [10]:
#Coal Input

base_coal_input = {
    "node_loc":country,
    "year_vtg":vintage_years,
    "year_act":act_years,
    "mode":"standard",
    "node_origin":country,
    "commodity":"coal",
    "time":"year",
    "time_origin":"year"
}

coal_bal = pd.DataFrame(
    dict(
        technology='coal_bal',
        level='primary',
        value = 1,
        unit="-",
        **base_coal_input,
    )
)

coal_t_d =pd.DataFrame(
    dict(
        technology='coal_t_d',
        level='secondary',
        value=1,
        unit="-",
        **base_coal_input,
    )
)

coal_ppl = pd.DataFrame(
    dict(
        technology='coal_ppl',
        level='primary',
        value=1,
        unit='-',
        **base_coal_input,
    )
)

scenario.add_par("input",coal_bal)
scenario.add_par("input",coal_t_d)
scenario.add_par("input",coal_ppl)



In [11]:
from message_ix import make_df

#Electricity Input
base_electr_input = {
    "node_loc":country,
    "year_vtg":vintage_years,
    "year_act":act_years,
    "mode":"standard",
    "node_origin":country,
    "commodity":"electricity",
    "time":"year",
    "time_origin":"year"
}

electr_grid = make_df(
    base_electr_input,
    technology='electricity_grid',
    level='secondary',
    value=1,
    unit='-',
)

scenario.add_par("input",electr_grid)



In [ ]:
coal_bal_eff = 0.9
coal_ppl_eff = 0.3
electr_grid_eff = 0.9
coal_t_d_eff = 0.9

#Output
base_output = {
'node_loc':country,
'year_vtg':vintage_years,
'year_act':act_years,
'mode':'standard',
'node_dest':country,
'time':'year',
'time_dest':'year',   
}

#coal_bal
coal_bal = make_df(
    base_output,
    technology ='coal_bal',
    commodity ='coal',
    level = 'secondary',
    value = 0.9, #Assumption to be changed (10% losses)
    unit = '-'
    
)

#coal_ppl
coal_ppl = make_df(
    base_output,
    technology ='coal_ppl',
    commodity ='electricity',
    level = 'secondary',
    value = 0.3, #Assumption to be changed
    unit = '-'
    
)

#coal_t_d
coal_t_d = make_df(
    base_output,
    technology = 'coal_t_d',
    commodity = 'coal',
    level='final',
    value=0.9, #Assumption to be changed
    unit = '-'
)

#electricity grid
electr_grid = make_df(
    base_output,
    technology ='electricity_grid',
    commodity ='electricity',
    level = 'final',
    value = 0.9, #Assumption to be changed
    unit = '-'
)

scenario.add_par("output",coal_bal)
scenario.add_par("output",coal_ppl)
scenario.add_par("output",coal_t_d)
scenario.add_par("output",electr_grid)



In [13]:
#Technical Lifetime (data from tutorial)

technical_lifetime = make_df("technical_lifetime",
    node_loc = country,
    year_vtg = vintage_years,
    unit = 'y',
    technology = 'coal_ppl',
    value = 40,
)

scenario.add_par('technical_lifetime',technical_lifetime)

In [ ]:
#Capacity Factor (data from tutorial)
coal_ppl_cf = 0.85

capacity_factor = make_df(
   'capacity_factor',
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    time = 'year',
    unit = '-',
    technology = 'coal_ppl',
    value = 0.85
)

scenario.add_par('capacity_factor',capacity_factor)

## Technoeconomic Parameters

In [15]:
#inv_cost (data from tutorial)

inv_cost = make_df(
    "inv_cost",
    node_loc = country,
    year_vtg = vintage_years,
    unit = 'USD/kW',
    technology = 'coal_ppl',
    value=1500,
)

mp.add_unit('USD/kW')

scenario.add_par('inv_cost',inv_cost)

In [16]:
#fix_cost (data from tutorial)

fix_cost = make_df(
    "fix_cost",
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    unit = 'USD/kWa',
    technology = 'coal_ppl',
    value=40,
)

mp.add_unit('USD/kWa')

scenario.add_par('fix_cost',fix_cost)

In [17]:
#var_cost (data from tutorial)

var_cost = make_df(
    "var_cost",
    node_loc = country,
    year_vtg = vintage_years,
    year_act = act_years,
    unit = 'USD/kWa',
    mode='standard',
    time='year',
    technology = 'coal_ppl',
    value=213.744
)

scenario.add_par('var_cost',var_cost)

In [20]:
horizon = list(range(2010, 2041, 10))
horizon

[2010, 2020, 2030, 2040]

In [21]:
horizon[1:]

[2020, 2030, 2040]

In [27]:
mp.close_db()

In [34]:
base_activity = {
    "node_loc": country,
    "year_act": [2010,2011],
    "mode": "standard",
    "time": "year",
    "unit": "GWa",
}

# in GWh - from IEA Electricity Output
activity = {
    "coal_ppl": [7184,1],
    "gas_ppl": [14346,1],
    "oil_ppl": [1275,1],
    "hydro_ppl": [38406,1],
    "bio_ppl": [4554,1],
    "wind_ppl": [2064,1],
    "solar_pv_ppl": [89,1],
    "import": [2340,1],
    "cfl": [0,1],
}

# MODEL CALIBRATION: by inserting an upper and lower bound to the same quantity we are
# ensuring that the model is calibrated at that value that year, so we are at the right
# starting point.

tes = pd.DataFrame(dict(**base_activity,**activity))
tes

,node_loc,year_act,mode,time,unit,coal_ppl,gas_ppl,oil_ppl,hydro_ppl,bio_ppl,wind_ppl,solar_pv_ppl,import,cfl
0,Indonesia,2010,standard,year,GWa,7184,14346,1275,38406,4554,2064,89,2340,0
1,Indonesia,2011,standard,year,GWa,1,1,1,1,1,1,1,1,1


In [35]:
scenario.idx_names('bound_activity_up')

['node_loc', 'technology', 'year_act', 'mode', 'time']

In [36]:
scenario.idx_names('bound_activity_lo')

['node_loc', 'technology', 'year_act', 'mode', 'time']

In [37]:
scenario.idx_names('historical_activity')

['node_loc', 'technology', 'year_act', 'mode', 'time']

In [38]:
scenario.idx_names('historical_new_capacity')

['node_loc', 'technology', 'year_vtg']

In [39]:
mp.close_db()